In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2

from keras.models import Sequential
import keras.layers
from sklearn.model_selection import train_test_split

In [2]:
# image functions
def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img
def toGray(im, im_L=10, im_W=10, contrast=1.0):
    # normalize
    gray = np.zeros([im_L, im_W])
    for j in range(im_L):
        for i in range(im_W):
            gray[j][i] = max(im[j][i]) * contrast
    gray = convert(gray, 0, 255, np.uint8)
    return gray

In [3]:
# import data and label data
sample_set = np.empty([120*9, 10, 10])
sample_labels = np.empty(120*9)
baseDir = 'ShapeGroups'
idx = 0
shape_id = 0
for filename in os.listdir(baseDir):
    folder = baseDir + '/' + filename
    for imagepath in os.listdir(folder):
        imagepathFull = folder + '/' + imagepath
        sample_set[idx] = toGray(cv2.imread(imagepathFull))
        sample_labels[idx] = shape_id
        # print("\n" + im
        # print(training_set[idx])
        idx += 1
    shape_id += 1
sample_set = sample_set[0:idx,]
sample_labels = sample_labels[0:idx,]

C:\Users\ereij\AppData\Local\Temp\ipykernel_9968\2359215898.py:6: RuntimeWarning: divide by zero encountered in scalar divide
  a = (target_type_max - target_type_min) / (imax - imin)
C:\Users\ereij\AppData\Local\Temp\ipykernel_9968\2359215898.py:8: RuntimeWarning: invalid value encountered in add
  new_img = (a * img + b).astype(target_type)
C:\Users\ereij\AppData\Local\Temp\ipykernel_9968\2359215898.py:8: RuntimeWarning: invalid value encountered in cast
  new_img = (a * img + b).astype(target_type)


In [4]:
# add padding - 11 rows and columns on each side for 32x32
sample_size = len(sample_set)
X_set = np.empty([sample_size, 32, 32])
for idx in range(sample_size):
    X_set[idx] = np.pad(sample_set[idx], ((11, 11), (11, 11)), 'constant', constant_values=(0, 0))

print(np.shape(X_set[0]))

(32, 32)


In [5]:
# Split Dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_set, sample_labels, test_size=0.33, random_state=42)

print(len(X_train))
print(len(X_test))
# print(Y_test)

719
355


In [8]:
# Set up CNN
model = Sequential()

# Convolution 1: Input = 32x32x1, Output = 28x28x6
# Subsampling 1: Input = 28x28x6, Output = 14x14x6
model.add(keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32,32,1)))
model.add(keras.layers.AveragePooling2D())

# Convolution 2: Input = 14x14x6, Output = 10x10x16
# Subsampling 2: Input = 10x10x16, Output = 5x5x16
model.add(keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.AveragePooling2D())

# Fully Connected Layers
model.add(keras.layers.Flatten())

# Input = 5x5x16, Output = 120
model.add(keras.layers.Dense(units=120, activation='relu'))
# Input = 120, Output = 84
model.add(keras.layers.Dense(units=84, activation='relu'))

# Softmax output - modified for 7 shapes
model.add(keras.layers.Dense(units=7, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 30, 30, 6)         60        
                                                                 
 average_pooling2d_2 (Avera  (None, 15, 15, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 16)        880       
                                                                 
 average_pooling2d_3 (Avera  (None, 6, 6, 16)          0         
 gePooling2D)                                                    
                                                                 
 flatten_1 (Flatten)         (None, 576)               0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [9]:
# Train the network
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the ANN model iteratively for n_epochs
n_epochs = 5
history = model.fit(x=X_train,y=Y_train, epochs=n_epochs, batch_size=128, validation_data=(X_test, Y_test))

Epoch 1/5
6/6 [==============================] - 2s 62ms/step - loss: 1.8115 - accuracy: 0.3839 - val_loss: 1.0532 - val_accuracy: 0.8845
Epoch 2/5
6/6 [==============================] - 0s 32ms/step - loss: 0.7958 - accuracy: 0.9597 - val_loss: 0.5934 - val_accuracy: 1.0000
Epoch 3/5
6/6 [==============================] - 0s 32ms/step - loss: 0.4775 - accuracy: 0.9972 - val_loss: 0.4333 - val_accuracy: 1.0000
Epoch 4/5
6/6 [==============================] - 0s 33ms/step - loss: 0.3585 - accuracy: 0.9972 - val_loss: 0.3684 - val_accuracy: 1.0000
Epoch 5/5
6/6 [==============================] - 0s 30ms/step - loss: 0.3109 - accuracy: 0.9972 - val_loss: 0.3299 - val_accuracy: 1.0000


In [10]:
def calc_Categorical_Accuracy(test, truth, num_categories):
    truth = truth.astype(int)
    certainty = model.predict(test, verbose=0)
    decision = np.argmax(certainty, axis=1)  # find index of max probability

    categorical_accuracy = np.zeros(shape=(num_categories,))
    for i in range(len(decision)):
      if decision[i] == truth[i]:
          categorical_accuracy[truth[i]] = categorical_accuracy[truth[i]]+1

    for i in range(num_categories):
        num_tests = truth.tolist().count(i)
        categorical_accuracy[i] = categorical_accuracy[i]/num_tests
    return categorical_accuracy

acc = calc_Categorical_Accuracy(X_test, Y_test, 7)
print(acc)

acc = calc_Categorical_Accuracy(X_train, Y_train, 7)
print(acc)

[1. 1. 1. 1. 1. 1. 1.]
[1.         1.         1.         1.         1.         0.97849462
 1.        ]
